## SQL запросы к Elasticsearch

In [4]:
import pandas as pd
import pandasticsearch as ps
from elasticsearch import Elasticsearch
import plotly.graph_objects as go
import plotly.express as px
import time
import os
import qgrid
from pprint import pprint, pp

Задаем конечную точку, текст SQL запроса и соединение с Elastics.

In [5]:
elastic_url = 'http://dockertest.rgwork.ru:9094/elasticsearch/' if os.path.exists('local-file.txt') else 'http://es01:9200/'
# print(elastic_url)
sql = '''
SELECT HISTOGRAM(date_modified, INTERVAL 1 HOUR) as interval, count(*) as count 
FROM articles 
GROUP by interval 
ORDER BY interval 
DESC LIMIT 100 
'''

ps_client = ps.RestClient(elastic_url, '_sql')

In [ ]:
def create_plot_widget():
    fig = px.bar(x=['a'], y=[1], 
                 title='Статей за час', 
                 template='plotly_white')
    fig.update_traces(marker_color='#3CC')
    fig.layout.yaxis.title.text = 'статей за час'
    fig.layout.xaxis.title.text = 'время'
    fig.update_xaxes(showgrid=True, gridwidth=1)
    fig.layout.title.font.size = 40
    fig.layout.title.x = 0.5
    fig.layout.title.y = 0.9
    return go.FigureWidget(fig)

def update_plot_widget(widg, xvalues, yvalues):
    bar = widg.data[0]
    bar.x = xvalues
    bar.y = yvalues
    bar.text = bar.y.tolist()
    
def update_plot_n_times(widg, sql, times=10, delay=5):
    if not widg:
        return
    
    for i in range(times):
        res = ps_client.post({'query':sql})
        df = pd.DataFrame.from_records(res['rows'],columns=[c['name'] for c in res['columns']])         
        update_plot_widget(widg, df['interval'], df['count'])
        widg.layout.title = f'Request {i+1}/{times}'
        time.sleep(delay)        
    
print(__name__)   
pw = create_plot_widget()
display(pw)

update_plot_n_times(pw, sql, times=100, delay=10)

FigureWidget({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'x=%{x}<br>y=%{y}<extra>…

In [ ]:
import multiprocessing
import time

# object to save module variables
HISTOGRAM = {
    'process': None
    'total':0,
    'totla_last_day':0,
}

# bar
def bar():
    for i in range(100):
        print(f"Tick {i}")
        time.sleep(10)

if __name__ == '__main__':
    # Start bar as a process
    p = multiprocessing.Process(target=bar)
    p.start()

    # Wait for 10 seconds or until process finishes
    print('joining')
    p.join(1)
    print('joined')

    # If thread is still active
    if p.is_alive():
        print("running... let's kill it...")

        # Terminate - may not work if process is stuck for good
        p.terminate()
        # OR Kill - will work for sure, no chance for process to finish nicely however
#         p.kill()

        p.join()
    print("alive" if p.is_alive() else "dead")

TODO
=====

- заменить запрос на нормальный
- обойтись без панды
- сделать кнопки пуска останова обновления диаграммы
- add counter annotation (всего и за прошедшие сутки)
- pack code into a class


In [12]:
%%html
<style>
h1,h2,h3,h4,h5 {
    /* color:gray;*/
    font-weight:bold !important;
}
</style>